In [1]:
import numpy as np
from pathlib import Path
import skimage as sk
import cv2 as cv
import skimage.io as skio
import skimage.color as skcolor
# import scipy.signal as scsignal
import matplotlib.pyplot as plt
from helper import get_points
import scipy.ndimage as ndi

def load_image(path, greyscale=True):
    im = skio.imread(path)
    im = sk.img_as_float(im)
    
    # remove alpha if its there
    if im.shape[-1] == 4:
        im = skcolor.rgba2rgb(im)

    if greyscale and im.ndim == 3:
        im = skcolor.rgb2gray(im)
    return im

def save_image(path, im):
    skio.imsave(path, sk.img_as_ubyte(im))

In [2]:
# Get Points
from helper import get_points

im1 = load_image('./images/set1/L1.jpg', greyscale=False)
im2 = load_image('./images/set1/L2.jpg', greyscale=False)

%matplotlib qt
points1, points2 = get_points(im1, im2)
%matplotlib inline



Please select 10 points in each image for alignment.


In [8]:
def computeH(im1_pts, im2_pts):
    A = []
    b = []
    for i in range(len(im1_pts)):
        x1, y1 = im1_pts[i]
        x2, y2 = im2_pts[i]
        A.append([x1, y1, 1, 0, 0, 0, -x1*x2, -y1*x2])
        A.append([0, 0, 0, x1, y1, 1, -x1*y2, -y1*y2])

        b.append(x2)
        b.append(y2)
    
    A = np.array(A)
    b = np.array(b)
    h = np.linalg.lstsq(A, b, rcond=None)[0]
    H = np.append(h, 1).reshape(3, 3)
    return H


def get_output_shape(im_to_warp, H):
    h, w = im_to_warp.shape[:2]

    corners = np.array([
        [0, 0, 1],
        [w, 0, 1],
        [w, h, 1],
        [0, h, 1]
    ]).T

    warped_corners = H @ corners
    warped_corners /= warped_corners[2,:]

    ref_corners = corners[:2, :]

    all_corners = np.hstack((warped_corners[:2,:], ref_corners))

    x_min, y_min = np.min(all_corners, axis=1)
    x_max, y_max = np.max(all_corners, axis=1)
    
    output_shape = (int(np.ceil(y_max - y_min)), int(np.ceil(x_max - x_min)))
    
    offset = np.array([
        [1, 0, -x_min],
        [0, 1, -y_min],
        [0, 0, 1]
    ])

    return output_shape, offset

def warpImageNearestNeighbor(im, H):
    output_shape, M_offset = get_output_shape(im, H)
    composite_H = M_offset @ H
    H_inv = np.linalg.inv(composite_H)
    output = np.zeros(output_shape + im.shape[2:], dtype=im.dtype)
    out_h, out_w = output_shape
    h, w = im.shape[:2]

    y_out, x_out = np.meshgrid(np.arange(out_h), np.arange(out_w), indexing='ij')

    out_coords = np.vstack((x_out.ravel(), y_out.ravel(), np.ones(out_h * out_w)))
    
    src_coords_h = H_inv @ out_coords

    w_coords = src_coords_h[2, :]
    w_coords[w_coords == 0] = 1e-8
    
    src_x = (src_coords_h[0, :] / w_coords).reshape(output_shape)
    src_y = (src_coords_h[1, :] / w_coords).reshape(output_shape)

    src_x_nn = np.round(src_x).astype(int)
    src_y_nn = np.round(src_y).astype(int)

    valid_mask = (src_x_nn >= 0) & (src_x_nn < w) & (src_y_nn >= 0) & (src_y_nn < h)

    output[valid_mask] = im[src_y_nn[valid_mask], src_x_nn[valid_mask]]
    
    return output

def warpImageBilinear(im, H):
    output_shape, M_offset = get_output_shape(im, H)
    out_h, out_w = output_shape[:2]
    h, w = im.shape[:2]
    
    # build in shift to map it for cropping
    composite_H = M_offset @ H
    H_inv = np.linalg.inv(composite_H)
    
    y_out, x_out = np.meshgrid(np.arange(out_h), np.arange(out_w), indexing='ij')
    out_coords = np.vstack((x_out.ravel(), y_out.ravel(), np.ones(out_h * out_w)))
    src_coords_h = H_inv @ out_coords # inverse map output coordinates
    src_x = (src_coords_h[0,:] / src_coords_h[2,:]).reshape(output_shape)
    src_y = (src_coords_h[1,:] / src_coords_h[2,:]).reshape(output_shape) # normalize them with the w
    
    # top left and bottom right
    x0 = np.floor(src_x).astype(int)
    y0 = np.floor(src_y).astype(int)
    x1 = x0 + 1
    y1 = y0 + 1
    
    # differences
    dx = src_x - x0
    dy = src_y - y0
    
    valid_points = (x0 >= 0) & (x1 < w) & (y0 >= 0) & (y1 < h)
    
    output = np.zeros(output_shape + im.shape[2:], dtype=im.dtype)

    for channel in range(im.shape[2]):
        im_ch = im[:,:,channel]
        
        # get all of the point mappings
        top_left = im_ch[y0[valid_points], x0[valid_points]]
        top_right = im_ch[y0[valid_points], x1[valid_points]]
        bottom_left = im_ch[y1[valid_points], x0[valid_points]]
        bottom_right = im_ch[y1[valid_points], x1[valid_points]]

        dx_masked = dx[valid_points]
        dy_masked = dy[valid_points]
        
        top_interp = top_left * (1 - dx_masked) + top_right * dx_masked
        bottom_interp = bottom_left * (1 - dx_masked) + bottom_right * dx_masked
        interpolated_values = top_interp * (1 - dy_masked) + bottom_interp * dy_masked
        
        channel_out = np.zeros(output_shape, dtype=im.dtype)
        channel_out[valid_points] = interpolated_values
        output[:,:,channel] = channel_out
        
    return output

In [ ]:
H = computeH(points1, points2)
print(H)

imwarped_nn = warpImageNearestNeighbor(im1,H)
imwarped_bil = warpImageBilinear(im1,H)

save_image('./images/set1/L1_warped.jpg', imwarped_nn)
save_image('./images/set1/L1_warped_bilinear.jpg', imwarped_bil)



[[ 2.37368465e+00 -5.64645767e-02 -4.44675883e+03]
 [ 5.84127459e-01  1.90611404e+00 -1.34499188e+03]
 [ 3.62312430e-04 -3.92715061e-05  1.00000000e+00]]


In [ ]:
def warpImageBilinear_rectify(im, H, output_shape):
    out_h, out_w = output_shape[:2]
    h, w = im.shape[:2]
    
    H_inv = np.linalg.inv(H)
    
    y_out, x_out = np.meshgrid(np.arange(out_h), np.arange(out_w), indexing='ij')
    out_coords = np.vstack((x_out.ravel(), y_out.ravel(), np.ones(out_h * out_w)))
    src_coords_h = H_inv @ out_coords
    src_x = (src_coords_h[0,:] / src_coords_h[2,:]).reshape(output_shape)
    src_y = (src_coords_h[1,:] / src_coords_h[2,:]).reshape(output_shape)
    
    x0 = np.floor(src_x).astype(int)
    y0 = np.floor(src_y).astype(int)
    x1 = x0 + 1
    y1 = y0 + 1
    
    dx = src_x - x0
    dy = src_y - y0
    
    valid_points = (x0 >= 0) & (x1 < w) & (y0 >= 0) & (y1 < h)
    
    output = np.zeros(output_shape + im.shape[2:], dtype=im.dtype)

    for channel in range(im.shape[2]):
        im_ch = im[:,:,channel]
        
        top_left = im_ch[y0[valid_points], x0[valid_points]]
        top_right = im_ch[y0[valid_points], x1[valid_points]]
        bottom_left = im_ch[y1[valid_points], x0[valid_points]]
        bottom_right = im_ch[y1[valid_points], x1[valid_points]]

        dx_masked = dx[valid_points]
        dy_masked = dy[valid_points]
        
        top_interp = top_left * (1 - dx_masked) + top_right * dx_masked
        bottom_interp = bottom_left * (1 - dx_masked) + bottom_right * dx_masked
        interpolated_values = top_interp * (1 - dy_masked) + bottom_interp * dy_masked
        
        channel_out = np.zeros(output_shape, dtype=im.dtype)
        channel_out[valid_points] = interpolated_values
        output[:,:,channel] = channel_out
        
    return output


to_rectify = load_image('./images/rectify/square3.jpg', greyscale=False)

%matplotlib qt
rectify_points = get_points(to_rectify)
%matplotlib inline

points = np.array(rectify_points)
top_width = np.linalg.norm(points[0] - points[1])
bottom_width = np.linalg.norm(points[3] - points[2])
avg_width = int(np.mean([top_width, bottom_width]))

left_height = np.linalg.norm(points[0] - points[3])
right_height = np.linalg.norm(points[1] - points[2])
avg_height = int(np.mean([left_height, right_height]))

print(f"Output size: {avg_width}w x {avg_height}h")
points_dst = np.array([
    [0, 0],
    [avg_width - 1, 0],
    [avg_width - 1, avg_height - 1],
    [0, avg_height - 1]
])

H_rectify = computeH(rectify_points, points_dst)
output_shape = (avg_height, avg_width)
rectified = warpImageBilinear_rectify(to_rectify, H_rectify, output_shape)

save_image('./images/rectify/square3_rectified.jpg', rectified)

Please select 10 points in each image for alignment.
Output size: 942w x 1437h
[[[0.29509803 0.21541889 0.13582887]
  [0.24491041 0.17238359 0.09492672]
  [0.21921966 0.14886885 0.07585861]
  ...
  [0.38111977 0.33013938 0.25690788]
  [0.38151748 0.33053709 0.25602729]
  [0.38595852 0.33497812 0.26046832]]

 [[0.23268232 0.16145794 0.08481914]
  [0.22691768 0.16142096 0.08454933]
  [0.23880276 0.17317784 0.10218119]
  ...
  [0.37926517 0.32828478 0.2540118 ]
  [0.38524861 0.33426822 0.25975842]
  [0.38989924 0.33891885 0.26440904]]

 [[0.22310688 0.15625041 0.08547172]
  [0.24101952 0.18089    0.10703319]
  [0.25475759 0.19187982 0.1290232 ]
  ...
  [0.39203635 0.34105596 0.26654616]
  [0.40129244 0.35031205 0.27580224]
  [0.39849938 0.34751898 0.27300918]]

 ...

 [[0.9369629  0.90559035 0.85460996]
  [0.9372549  0.90588235 0.85490196]
  [0.93921914 0.90784659 0.8568662 ]
  ...
  [0.37237856 0.32531974 0.27041778]
  [0.37293652 0.3258777  0.27097574]
  [0.37588823 0.3288294  0.2739274